References:
* OpenAI's API: https://platform.openai.com/docs/guides/gpt/function-calling
* Functions Cookbook: https://github.com/openai/openai-cookbook/blob/main/examples/How_to_call_functions_with_chat_models.ipynb

In [3]:
# This notebook tests the new function calling feature of GPT-3.5-turbo.

# Set up chatgpt api
import openai
import json
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
GPT_KEY = os.environ['OPENAI_API_KEY']
GPT_MODEL_OLD = "gpt-3.5-turbo"
GPT_MODEL = "gpt-3.5-turbo-0613"


# 0. The problem of unstructured answers

LLMs ingeneral and ChatGPT in particular only aim to generate the next most feasible token. This has made it notoriously hard to incorporate LLMs with existing applications which require structured data.

## 1. Parse information

In [5]:
# messages=[{"role": "user", "content": "Hi, can I buy three Apple Pies?"}],
messages=[{"role": "user", "content": "Hi, can I buy half a pound of salmon?"}],

In [31]:
def parser_without_functions(message):
    """
    Parse the product name, quantity, and unit from the user's message.
    """
    gpt_response = openai.ChatCompletion.create(
        model=GPT_MODEL_OLD,
        messages=[
            {"role": "system", "content": "Parse the product name, quantity, and unit from the user's message. Return ONLY a JSON object with the product_name, product_quantity, and unit. Do not include any other information."},
            {"role": "user", "content": message}
            ],
    )
    data = gpt_response.choices[0]["message"]["content"]
    try:
        data = json.loads(data)
    except:
        # print("Info: cannot parse data from gpt_response.")
        pass
    return data, gpt_response

def parser_with_functions(message):
    """
    Parse the product name, quantity, and unit from the user's message.
    """
    gpt_response = openai.ChatCompletion.create(
        model=GPT_MODEL,
        messages=[{"role": "user", "content": message}],
        functions=[
            {
                "name": "get_product_info",
                "description": "Parse the product name, quantity, and unit from the user's message.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "prduct_name": {
                            "type": "string",
                            "description": "The name of the product."
                            },
                        "product_quantity": {
                            "type": "number", 
                            "description": "The quantity of the product."
                            },
                        "unit": {
                            "type": "string", 
                            "description": "The unit of the product.", 
                            "enum": ["kg", "g", "lb", "oz", "unit"]
                            },
                    },
                    "required": ["prduct_name", "product_quantity", "unit"],
                },
            }
        ],
        function_call={"name": "get_product_info"},
    )
    data_json = gpt_response.choices[0]["message"]['function_call']['arguments']
    data = json.loads(data_json)
    return data, gpt_response

In [32]:
data, gpt_response = parser_without_functions("Hi, can I buy half a pound of salmon?")
print(data)

Sure! Here is the JSON object with the relevant information:

```
{
  "product_name": "salmon",
  "product_quantity": "0.5",
  "unit": "pound"
}
```


In [25]:
data, gpt_response = parser_with_functions("Hi, can I buy half a pound of salmon?")
print(data)
print(gpt_response)

{'product_name': 'salmon', 'product_quantity': 0.5, 'unit': 'lb'}
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": null,
        "function_call": {
          "arguments": "{\n  \"product_name\": \"salmon\",\n  \"product_quantity\": 0.5,\n  \"unit\": \"lb\"\n}",
          "name": "get_product_info"
        },
        "role": "assistant"
      }
    }
  ],
  "created": 1687128773,
  "id": "chatcmpl-7SvYLG81xG97roDBhs5Auonipaqob",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 28,
    "prompt_tokens": 122,
    "total_tokens": 150
  }
}


In [29]:
data, gpt_response = parser_without_functions("Hi, can I buy half a pound of salmon?")
print(data)
print(gpt_response)

Info: cannot parse data from gpt_response.
Sure! Here's the JSON object for that request:

```
{
  "product_name": "salmon",
  "product_quantity": 0.5,
  "unit": "pound"
}
```
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Sure! Here's the JSON object for that request:\n\n```\n{\n  \"product_name\": \"salmon\",\n  \"product_quantity\": 0.5,\n  \"unit\": \"pound\"\n}\n```",
        "role": "assistant"
      }
    }
  ],
  "created": 1687128908,
  "id": "chatcmpl-7SvaWlg95ub0DTIiELqnQGzuIODki",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 43,
    "prompt_tokens": 55,
    "total_tokens": 98
  }
}


## 2. Get structured answers from ChatGPT

In [6]:
data = []
with open("data/MultiRC/val.jsonl", "r") as f:
    data_in = f.readlines()
for line in data_in:
    data.append(json.loads(line))

In [7]:
content_good = """Text:
[beginning of text]
Einstein married Elsa Lowenthal on 2 June 1919, after having had a relationship with her since 1912. In 1933, they emigrated to the United States. In 1935, Elsa Einstein was diagnosed with heart and kidney problems; she died in December 1936. 
[end of text]
Summary:
[beginning of summary]
Questions: Where was Elsa Einstein most likely living when she was diagnosed with heart and kidney problems?
Answer: United States
[end of summary]
"""
content_bad = """Text:
[beginning of text]
Einstein married Elsa Lowenthal on 2 June 1919, after having had a relationship with her since 1912. In 1933, they emigrated to the United States. In 1935, Elsa Einstein was diagnosed with heart and kidney problems; she died in December 1936. 
[end of text]
Summary:
[beginning of summary]
Questions: Where was Elsa Einstein most likely living when she was diagnosed with heart and kidney problems?
Answer: In Zurich
[end of summary]
"""
# In Einstein's heart
# ASSESSMENT_CHOICES = ["contradiction", "entailment", "neutral"]
ASSESSMENT_CHOICES = ["contradiction", "entailment"]

In [8]:
completion = openai.ChatCompletion.create(
    model=GPT_MODEL,
    messages=[{"role": "user", "content": content_good}],
    functions=[
    {
        "name": "check_contradiction",
        "description": "Check the contradiction between the text and the summary",
        "parameters": {
            "type": "object",
            "properties": {
                # "analysis": {
                #     "type": "string",
                #     "description": "A logical analysis comparing the text and the summary",
                # },
                "assessment": {
                    "type": "string", 
                    "description": "Final assessment of the contradiction between the text and the summary",
                    "enum": ASSESSMENT_CHOICES
                },
            },
            # "required": ["analysis", "assessment"],
            "required": ["assessment"],
        },
    }
],
function_call="auto",
)
args = json.loads(completion["choices"][0]["message"].to_dict()['function_call']["arguments"])
args

{'assessment': 'entailment'}

In [9]:
completion = openai.ChatCompletion.create(
    model=GPT_MODEL,
    messages=[{"role": "user", "content": content_bad}],
    functions=[
    {
        "name": "check_contradiction",
        "description": "Check the contradiction between the text and the summary",
        "parameters": {
            "type": "object",
            "properties": {
                # "analysis": {
                #     "type": "string",
                #     "description": "A short analysis of the text and the summary",
                # },
                "assessment": {"type": "string", "enum": ASSESSMENT_CHOICES},
            },
            "required": ["analysis", "assessment"],
        },
    }
],
function_call="auto",
)
args = json.loads(completion["choices"][0]["message"].to_dict()['function_call']["arguments"])
args

{'assessment': 'contradiction'}

# Use external tools

ChatGPT is powerful, however, it is still limited in many ways like not having access to the internet, limited ability to do math. Using functions, we can enable ChatGPT to use external tools to supplement its capabilities. In fact, this is probably the main use of functions that OpenAI's developer had in mind.

In [10]:
import sqlite3
from pprint import pprint

conn = sqlite3.connect('data/sqlite/grocery_txn.db')
conn.execute("""CREATE TABLE IF NOT EXISTS transactions (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_name TEXT NOT NULL,
    product_name TEXT NOT NULL,
    quantity INTEGER NOT NULL,
    price REAL NOT NULL,
    total REAL NOT NULL
);""")
rows = [
    ('john', 'apple', 10, 1.0, 10.0),
    ('john', 'orange', 5, 0.5, 2.5),
    ('hannah', 'apple', 5, 1.0, 5.0),
    ('hannah', 'orange', 10, 0.5, 5.0),
    ('hannah', 'banana', 5, 0.2, 1)]
conn.executemany("""INSERT INTO transactions (user_name, product_name, quantity, price, total)
    VALUES (?, ?, ?, ?, ?)""", rows)
conn.commit()

pprint(conn.execute("SELECT * FROM transactions").fetchall())
conn.execute("SELECT user_name, sum(total) as spending FROM transactions group by 1").fetchall()

[(1, 'john', 'apple', 10, 1.0, 10.0),
 (2, 'john', 'orange', 5, 0.5, 2.5),
 (3, 'hannah', 'apple', 5, 1.0, 5.0),
 (4, 'hannah', 'orange', 10, 0.5, 5.0),
 (5, 'hannah', 'banana', 5, 0.2, 1.0)]


[('hannah', 11.0), ('john', 12.5)]

In [11]:
def get_table_names(conn):
    """Return a list of table names."""
    table_names = []
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    for table in tables.fetchall():
        table_names.append(table[0])
    return table_names


def get_column_names(conn, table_name):
    """Return a list of column names."""
    column_names = []
    columns = conn.execute(f"PRAGMA table_info('{table_name}');").fetchall()
    for col in columns:
        column_names.append(col[1])
    return column_names


def get_database_info(conn):
    """Return a list of dicts containing the table name and columns for each table in the database."""
    table_dicts = []
    for table_name in get_table_names(conn):
        columns_names = get_column_names(conn, table_name)
        table_dicts.append({"table_name": table_name, "column_names": columns_names})
    return table_dicts

database_schema_dict = get_database_info(conn)
database_schema_string = "\n".join(
    [
        f"Table: {table['table_name']}\nColumns: {', '.join(table['column_names'])}"
        for table in database_schema_dict
    ]
)
print(database_schema_string)

Table: transactions
Columns: id, user_name, product_name, quantity, price, total
Table: sqlite_sequence
Columns: name, seq


In [12]:
database_schema_string = """Table: transactions
Columns: id, user_name, product_name, quantity, price, total"""

In [13]:
functions = [
    {
        "name": "ask_database",
        "description": "Use this function to answer user questions. Output should be a fully formed SQL query.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": f"""
                            SQL query extracting info to answer the user's question.
                            SQL should be written using this database schema:
                            {database_schema_string}
                            The query should be returned in plain text, not in JSON.
                            """,
                }
            },
            "required": ["query"],
        },
    },
]

def ask_database(conn, query):
    """Function to query SQLite database with a provided SQL query."""
    try:
        results = str(conn.execute(query).fetchall())
    except Exception as e:
        results = f"query failed with error: {e}"
    return results

def execute_function_call(message):
    if message["function_call"]["name"] == "ask_database":
        query = json.loads(message["function_call"]["arguments"])["query"]
        results = ask_database(conn, query)
    else:
        results = f"Error: function {message['function_call']['name']} does not exist"
    return results




In [14]:
messages = []
messages.append({"role": "system", "content": "Answer user questions by generating SQL queries against the Grocery Transaction Database."})
messages.append({"role": "user", "content": "Hi, how many apple was sold?"})


In [15]:
def generate_convo(messages, max_internal_turns=10):
    i = 0
    while not (messages[-1]["role"] == "assistant" and  not messages[-1].get("function_call")):
        chat_response = openai.ChatCompletion.create(
            model=GPT_MODEL,
            messages=messages,
            functions=functions,
        function_call="auto",
        )
        assistant_message = chat_response.to_dict()["choices"][0]["message"]
        messages.append(assistant_message)
        if assistant_message.get("function_call"):
            results = execute_function_call(assistant_message)
            messages.append({"role": "function", "name": assistant_message["function_call"]["name"], "content": results})
        i += 1
        if i > max_internal_turns:
            break
    return messages


In [16]:
generate_convo(messages)    
pprint(messages)


[{'content': 'Answer user questions by generating SQL queries against the '
             'Grocery Transaction Database.',
  'role': 'system'},
 {'content': 'Hi, how many apple was sold?', 'role': 'user'},
 {'content': None,
  'function_call': {'arguments': '{\n'
                                 '  "query": "SELECT SUM(quantity) FROM '
                                 'transactions WHERE product_name = '
                                 '\'apple\'"\n'
                                 '}',
                    'name': 'ask_database'},
  'role': 'assistant'},
 {'content': '[(15,)]', 'name': 'ask_database', 'role': 'function'},
 {'content': 'A total of 15 apples were sold.',
  'role': 'assistant'}]


In [17]:
# We can choose to expose only actual results and hide the function calls
# Also, we can add some color coding to the output
def pprint_convo(messages):
    CYAN = "\033[96m"
    GREEN = '\033[92m'
    RED = '\033[31m'
    BOLD = "\033[1m"
    col_role = {"user":CYAN, "assistant":GREEN}
    end_role = {"user":"", "assistant":"\n"}
    for mes in messages:
        role = mes["role"]
        cont = mes["content"]
        if mes["role"] not in ["user", "assistant"] or mes.get("function_call"):
            continue            
        else:
            # print("{role}: {content}".format(role=role, content=cont))
            print(f"""{col_role[role] + BOLD}{role}:\033[0m {col_role[role]}{cont}\033[0m{end_role[role]}""")
            

In [18]:
pprint_convo(messages)

user: Hi, how many apple was sold?
assistant: A total of 15 apples were sold.



In [19]:
messages.append({"role": "user", "content": "Who are the top 2 buying users in terms of number of unique products?"})
generate_convo(messages)
pprint_convo(messages)


user: Hi, how many apple was sold?
assistant: A total of 15 apples were sold.

user: Who are the top 2 buying users in terms of number of unique products?
assistant: The top 2 buying users in terms of the number of unique products are:
1. Hannah - She bought 3 unique products.
2. John - He bought 2 unique products.



In [20]:
messages.append({"role": "user", "content": "What product did hannah bought but john didn't?"})
generate_convo(messages)
pprint_convo(messages)


user: Hi, how many apple was sold?
assistant: A total of 15 apples were sold.

user: Who are the top 2 buying users in terms of number of unique products?
assistant: The top 2 buying users in terms of the number of unique products are:
1. Hannah - She bought 3 unique products.
2. John - He bought 2 unique products.

user: What product did hannah bought but john didn't?
assistant: Hannah bought the product "banana" that John didn't buy.

